In [44]:
# 패키지
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 데이터 불러오기
path = "~/Dacon/서울시 자치구별 도보 네트워크 공간정보.csv"
df = pd.read_csv(path, encoding="cp949")
df['geometry'] = df['링크 WKT'].copy()
df.drop(columns=['링크 WKT'], inplace=True)

print("Dimension of Data: ",df.shape)

# 공간정보 열에 NA가 있는 행 삭제
df = df.dropna(subset=['geometry'])

# 전체 도로 중 보행 가능만 필터
df['링크 유형 코드'] = df['링크 유형 코드'].astype(str).str[0:4]

txt = """1000
1001
1010
1011
1100
1101
1110
1111
0001
0010
0011"""
walker_LINK_CODE = txt.split('\n')
df = df[df["링크 유형 코드"].isin(walker_LINK_CODE)]

print("보행 가능 필터 후: ",df.shape)

# 공간정보 데이터로 바꾸기 gdf
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="epsg:4326")

Dimension of Data:  (491082, 22)
보행 가능 필터 후:  (278763, 22)


In [45]:
# 패키지
import pandas as pd
import folium
from folium import plugins


# 데이터 양이 많아 남서쪽만 필터
print(pd.DataFrame(df['시군구명'].value_counts()).sort_index().reset_index())

print("남서쪽 필터 전: ",gdf.shape[0])
gdf = gdf[gdf['시군구명'].isin(['양천구', '구로구', '영등포구', '동작구', "관악구", '금천구'])]
print("남서쪽 필터 후: ",gdf.shape[0])

########################## 지도 시각화
M = folium.Map(location=[37.484208,126.929676], zoom_start=17)

# GeoDataFrame의 각 LINESTRING을 지도에 추가
for idx, row in gdf.iterrows():
    # LINESTRING 좌표 추출
    coordinates = [(y, x) for x, y in row.geometry.coords]

    # LineString 추가
    folium.PolyLine(
        locations=coordinates,
        weight=2,
        color='blue',
        opacity=0.8,
        popup=f"LineString ID: {idx}"  # 팝업 정보 (필요에 따라 수정)
    ).add_to(M)

M.save("seoul_south_west.html")

    시군구명  count
0    강남구  16863
1    강동구  11028
2    강북구  10780
3    강서구  10856
4    관악구  11137
5    광진구   9351
6    구로구  11426
7    금천구   7045
8    노원구  13553
9    도봉구   8418
10  동대문구  12412
11   동작구  10232
12   마포구  11657
13  서대문구   9507
14   서초구  10881
15   성동구   9711
16   성북구  15135
17   송파구  12625
18   양천구  11111
19  영등포구  13126
20   용산구   9879
21   은평구  11375
22   종로구  10706
23    중구   7771
24   중랑구  12178
남서쪽 필터 전:  278763
남서쪽 필터 후:  64077


In [35]:
gdf

,노드링크 유형,노드 WKT,노드 ID,노드 유형 코드,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,링크 길이,시군구코드,...,읍면동명,고가도로,지하철네트워크,교량,터널,육교,횡단보도,"공원,녹지",건물내,geometry
270427,LINK,NaN,0,NaN,245801,1111,107014.0,26877.0,38.522,1147000000,...,목동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.87834 37.53243, 126.87849 37.5..."
270428,LINK,NaN,0,NaN,96192,1011,63622.0,63624.0,36.288,1147000000,...,신정동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.85829 37.51651, 126.85868 37.5..."
270429,LINK,NaN,0,NaN,219754,1111,57005.0,57006.0,40.605,1147000000,...,신월동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.84619 37.52200, 126.84651 37.5..."
270430,LINK,NaN,0,NaN,98970,1111,47708.0,48407.0,54.773,1147000000,...,목동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.86894 37.53360, 126.86903 37.5..."
270431,LINK,NaN,0,NaN,208795,1111,25540.0,25679.0,10.509,1147000000,...,목동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.88157 37.54156, 126.88155 37.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401816,LINK,NaN,0,NaN,33770,1111,120237.0,83492.0,26.183,1162000000,...,봉천동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.95372 37.48563, 126.95401 37.4..."
401907,LINK,NaN,0,NaN,39682,1111,120019.0,77465.0,57.521,1162000000,...,봉천동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.95490 37.47926, 126.95490 37.4..."
401920,LINK,NaN,0,NaN,32484,1111,54459.0,49549.0,65.472,1162000000,...,신림동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.90109 37.48204, 126.90113 37.4..."
401942,LINK,NaN,0,NaN,32490,1111,116044.0,116272.0,75.600,1162000000,...,봉천동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.94081 37.48515, 126.94153 37.4..."


In [36]:
import folium
from folium import plugins

M = folium.Map(location=[37.484208,126.929676], zoom_start=17)

# GeoDataFrame의 각 LINESTRING을 지도에 추가
for idx, row in gdf.iterrows():
    # LINESTRING 좌표 추출
    coordinates = [(y, x) for x, y in row.geometry.coords]

    # LineString 추가
    folium.PolyLine(
        locations=coordinates,
        weight=2,
        color='blue',
        opacity=0.8,
        popup=f"LineString ID: {idx}"  # 팝업 정보 (필요에 따라 수정)
    ).add_to(M)

M.save("seoul_south_west.html")

In [9]:
pointdata = pd.DataFrame([{"name":"신림역", "geometry":"POINT (126.9296 37.4841)"}])

pointdata['geometry'] = pointdata['geometry'].apply(wkt.loads)
pointdata = gpd.GeoDataFrame(pointdata, geometry='geometry', crs="epsg:4326")

print(pointdata)
print("="*90)

pointdata_buffer = pointdata.to_crs(epsg=3857).geometry.buffer(1000)
pointdata_buffer = pointdata_buffer.to_crs(epsg=4326)
print(pointdata_buffer)

  name                    geometry
0  신림역  POINT (126.92960 37.48410)
0    POLYGON ((126.93858 37.48410, 126.93854 37.483...
dtype: geometry


In [10]:
from folium import GeoJson

M = folium.Map(location=[37.484165, 126.929699], zoom_start=16)


GeoJson(pointdata_buffer,style_function=lambda x: {
    'fillColor': 'blue',
    'fillOpacity': 0.3,
    'color': 'blue',
    'weight': 0.5
}).add_to(M)

M

In [11]:
intersect_sinlim = gdf[gdf.intersects(pointdata_buffer.geometry.iloc[0])]
# intersect_sinlim

In [12]:
import folium
from folium import Marker


M = folium.Map(location=[37.484165, 126.929699], zoom_start=16, tiles='cartodbpositron')

# GeoDataFrame의 각 LINESTRING을 지도에 추가
for idx, row in intersect_sinlim.iterrows():
    # LINESTRING 좌표 추출
    if row.geometry.geom_type == 'LineString':
        coordinates = [(y, x) for x, y in row.geometry.coords]
    elif row.geometry.geom_type == 'MultiLineString':
        coordinates = [(y, x) for line in row.geometry.geoms for x, y in line.coords]
    else:
        continue  # LINESTRING이나 MultiLineString이 아닌 경우 건너뛰기

    # LineString 추가
    folium.PolyLine(
        locations=coordinates,
        weight=2,
        color='red',
        opacity=0.8,
        popup=f"LineString ID: {idx}"  # 팝업 정보 (필요에 따라 수정)
    ).add_to(M)
# 지도 저장
M